# introduction

input으로 sequence가 들어가면 <sos>를 포함해서 word 하나하나가 $x_t$ 가 되게 되고<br/>
***$X = \{x_1, x_2, ..., x_T\}$ <br/> ***
RNN이든 LSTM이든 encoder를 거쳐서 $h_{t-1}$  부터 update되어 최종 context <br/>vector인 entire sentence에 대한 representation인  $h_T = z$ 가 생성된다.

$s_t = \text{DecoderRNN}(y_t, s_{t-1})$  

Decoder에는 마찬가지로 current word인 $y_t$와 직전시점 hidden state인 $s_{t-1}$이 들어가 $s_t$를 뱉는다. <br/> $s_0 = z = h_T$ 이며 인코더의 최종 은닉상태가 디코더의 initial state가 된다.

# preparing Data

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchtext.datasets import TranslationDataset, Multi30k
from torchtext.data import Field, BucketIterator

import spacy

import random  
import math
import time


In [2]:
SEED = 1234

random.seed(SEED)
torch.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

그다음으로 tokenizer를 생성한다. tokenizer란 sentence 같은 sring을 개별 token으로 분해시켜주는 것을 뜻한다 "good morning!" 이면 ["good","morning","!"]
spaCy는 각각의 언어("de"는 독일어, "en"은 영어)에 대하여 tokenize를 가능하게한다.<br/> #모델이 사전에 다운로드되어야함

In [3]:
spacy_de = spacy.load('de')
spacy_en = spacy.load('en')

이제 실제로 tokenize를 진행할 함수를 생성한다. 이 함수는 TorchText를 이용하여 문장을 받아 개별 토큰으로 반환한다. <br/>
해당 논문에서는 "최적화 문제를 해결하는데 도움을 주는 단기 의존성을 도입하기 위해" 인풋의 문장 순서를 뒤바꾼다.

In [4]:
def tokenize_de(text):
    """
    Tokenizes German text from a string into a list of strings (tokens) 
    and reverses it
    """
    return [tok.text for tok in spacy_de.tokenizer(text)][::-1]

def tokenize_en(text):
    """
    Tokenizes English text from a string into a list of strings (tokens)
    """
    return [tok.text for tok in spacy_en.tokenizer(text)]

In [5]:
SRC = Field(tokenize = tokenize_de, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

TRG = Field(tokenize = tokenize_en, 
            init_token = '<sos>', 
            eos_token = '<eos>', 
            lower = True)

이제 데이터셋을 다운로드하자. 우리가 쓰려는 Multi30k dataset은 30,000개까지의 영어, 독일어 그리고 불어 문장들을 내포한다. 

In [6]:
train_data , valid_data, test_data = Multi30k.splits(exts = 
('.de','.en'),
                                                            fields
= (SRC , TRG))

In [7]:
print(f"Number of training examples: {len(train_data.examples)}")
print(f"Number of validation examples: {len(valid_data.examples)}")
print(f"Number of testing examples: {len(test_data.examples)}")

Number of training examples: 29000
Number of validation examples: 1014
Number of testing examples: 1000


문장순서가 바뀌었는지 확인해보기 위해 예시를 한번 프린트해보자

In [8]:
print(vars(train_data.examples[0]))

{'src': ['.', 'büsche', 'vieler', 'nähe', 'der', 'in', 'freien', 'im', 'sind', 'männer', 'weiße', 'junge', 'zwei'], 'trg': ['two', 'young', ',', 'white', 'males', 'are', 'outside', 'near', 'many', 'bushes', '.']}


이제 source와 target 언어에 대한 vocabulary생성을 해보자. vocabulary는 각각의
고유한 토큰을 인덱스를 매겨서 원핫 인코딩을 하는데 쓰인다. source와 target 언어의 단어군은 서로 별개이다. 

min_freq를 사용해서 최소 두 번 이상 등장하는 단어를 어휘(vocab)에 넣고 한번씩 나오는 단어에 대해서는 <unk>토큰으로 처리한다.
    
단어군은 오로지 훈련 셋에서만 만들어져야한다(validation이나 test제외)

In [9]:
SRC.build_vocab(train_data, min_freq = 2)
TRG.build_vocab(train_data, min_freq = 2)

In [10]:
print(f'Unique Tokens in source (de) vocabulary: {len(SRC.vocab)}')
print(f'Unique Tokens in source (en) vocabulary: {len(TRG.vocab)}')

Unique Tokens in source (de) vocabulary: 7853
Unique Tokens in source (en) vocabulary: 5893


데이터 전처리의 마지막 단계는 이터레이터를 만드는 것이다. <br/>
또한 torch.device를 정의하여 TorchText가 gpu를 쓰게 할 것인지 말 것인지를 결정해야한다. 이터레이터를 통하여 예시의 배치들을 돌리려면 source 문장의 길이가같도록 패딩되어야  한다.

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
BATCH_SIZE = 128

train_iterator, valid_iterator, test_iterator = BucketIterator.splits(
    (train_data, valid_data, test_data), 
    batch_size = BATCH_SIZE, 
    device = device)

# Building the seq2seq Model

### Encoder

인코더는 두 개의 층을 가진 LSTM이다. 논문에서는 4개의 층을 쓰지만 여기서는 훈련 시간떄문에 두 개만 쓰도록 하자 

멀티레이어RNN에서 인풋 문장은 첫번째 bottom 레이어의 RNN과 은닉상태에 들어가 그 output이 다음 레이어의 인풋으로 들어간다. 첫번째 레이어 $h_t^1 = \text{EncoderRNN}^1(x_t, h_{t-1}^1)$ <br/>
두번쨰 레이어는 $h_t^2 = \text{EncoderRNN}^2(h_t^1, h_{t-1}^2)$ <br/>
그말은 $h_0^l$과 맥락 벡터인 $z^l$이 필요하단 소리다

RNN과 달리 LSTM은 단지 hidden state뿐만 아니라 cell state도 반환한다.

따라서 LSTM에 적용한 식은 다음과 같다.

$$\begin{align*}
(h_t^1, c_t^1)= \text{EncoderLSTM}^1(x_t, (h_{t-1}^1, c_{t-1}^1))\\
(h_t^2, c_t^2)= \text{EncoderLSTM}^2(h_t^1, (h_{t-1}^2, c_{t-1}^2))
\end{align*}$$ 

forward 방법에서, 임베딩 레이어를 통해 밀집된 벡터로 변환되는 source 문장 X를 인풋으로 넣으면 드롭아웃이 적용된다. 이 임베딩벡터들은 그다음에 RNN을 통과한다. 전체 시퀀스를 RNN에 넣기 때문에 hidden states에 대한 계산이 자동적으로된다. 여기서 초기 states를 RNN에 넣지 않는데 그럴때 자동적으로 제로 텐서로 생성하기 때문이다.(값이 없음)

RNN은 output, hidden($h_T$), 그리고 cell($c_T$)을 반환한다.

In [13]:
class Encoder(nn.Module):
    def __init__(self, input_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.input_dim = input_dim
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(input_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim, n_layers, dropout = dropout)
        
        self.dropout = nn.Dropout(dropout)
        
    def forward(self, src):
        
        #src = [src sent len, batch size]
        
        embedded = self.dropout(self.embedding(src))
        
        #embedded = [src sent len, batch size, emb dim]
        
        outputs, (hidden,cell) = self.rnn(embedded)
        
         #outputs = [src sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #outputs are always from the top hidden layer
        
        return hidden, cell

# Decoder

인코더와 마찬가지로 디코더의 레이어는 두 층의 LSTM이다.
첫 번 째 레이어는 그 전 시점으로부터 $(s_{t-1}^1, c_{t-1}^1)$ 를 받아서 현재 토큰인 $y_t$ 와 함께 첫 번째 LSTM에 넣어 나온 $(s_t^1, c_t^1)$ 과 직전 state인 $(s_{t-1}^2, c_{t-1}^2)$ 를 통해 $(s_t^2,c_t^2)$ 를 구할 수 있다

RNN에서 도출한 $s_t^L$를 이제 선형함수 $f$ 를 통과시켜서 타켓 함수의 다음 토큰에 대한 예측값인 $ \hat{y}_{t+1}$를 구한다 

인코더 클래스와 마찬가지로 디코더의 인풋에 들어가는 원핫벡터들의 사이즈의 output_dim이 있으며 추가적으로 hidden state가 들어가는 Linear layer이 있다.

forward방법에서 input 토큰과 이전 은닉상태와 셀 상태를 배치로 구성한다. 여기서 입력 토큰들을 1차원 문장의 길이로 만들어(조건) 임베딩 레이어를 통과시키고 드롭아웃을 시행한다.

In [14]:
class Decoder(nn.Module):
    def __init__(self, output_dim, emb_dim, hid_dim, n_layers, dropout):
        super().__init__()
        
        self.emb_dim = emb_dim
        self.hid_dim = hid_dim
        self.output_dim = output_dim
        self.n_layers = n_layers
        self.dropout = dropout
        
        self.embedding = nn.Embedding(output_dim, emb_dim)
        
        self.rnn = nn.LSTM(emb_dim, hid_dim , n_layers , dropout= dropout)
        self.out = nn.Linear(hid_dim, output_dim)
        
    def forward(self, input, hidden, cell):
        #input = [batch size]
        #hidden = [n layers* n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #n directions in the decoder will both always be 1, therefore:
        #hidden = [n layers, batch size, hid dim]
        #context = [n layers, batch size, hid dim]
        
        input = input.unsqueeze(0)
        
        #input = [1, batch size]
        
        embedded = self.dropout(self.embedding(input))
        
        #embedded = [1, batch size, emb dim]
                
        output, (hidden, cell) = self.rnn(embedded, (hidden, cell))
        
        #output = [sent len, batch size, hid dim * n directions]
        #hidden = [n layers * n directions, batch size, hid dim]
        #cell = [n layers * n directions, batch size, hid dim]
        
        #sent len and n directions will always be 1 in the decoder, therefore:
        #output = [1, batch size, hid dim]
        #hidden = [n layers, batch size, hid dim]
        #cell = [n layers, batch size, hid dim]
        
        prediction = self.out(output.squeeze(0))
        
        #prediction = [batch size, output dim]
        
        return prediction, hidden, cell

# Seq2Seq

seq2seq 모델은 인코더 , 디코더, 그리고 device를 포함한다. <br/>
실행을 위해서는 인코더와 디코더의 히든,셀 레이어의 수가 같아야 한다. 

In [15]:
class Seq2Seq(nn.Module):
    def __init__(self, encoder, decoder, device):
        super().__init__()
        
        self.encoder = encoder
        self. decoder = decoder
        self.device = device
        
        assert encoder.hid_dim == decoder.hid_dim, \
            "Hidden dimensions of encoder and decoder must be equal!"
        assert encoder.n_layers == decoder.n_layers, \
            "Encoder and decoder must have equal number of layers!"
    
    def forward(self, src, trg, teacher_forcing_ratio = 0.5): 
        
        #src = [src sent len, batch size]
        #trg = [trg sent len, batch size]
        #teacher_forcing_ratio is probability to use teacher forcing
        #e.g. if teacher_forcing_ratio is 0.75 we use ground-truth inputs 75% of the time
        
        batch_size = trg.shape[1]
        max_len = trg.shape[0]
        trg_vocab_size = self.decoder.output_dim 
        
        #tensor to store decoder outputs
        outputs = torch.zeros(max_len, batch_size, trg_vocab_size).to(self.device)
        
        #last hidden state of the encoder is used as the initial hidden state of the decoder
        hidden, cell = self.encoder(src)
        
        #first input to the decoder is the <sos> tokens
        input = trg[0,:]
        
        for t in range(1, max_len):
            
            output, hidden, cell = self.decoder(input, hidden, cell)
            outputs[t] = output
            teacher_force = random.random() < teacher_forcing_ratio
            top1 = output.max(1)[1]
            input = (trg[t] if teacher_force else top1)
        
        return outputs

# Training the Seq2Seq Model

이제 모델을 학습시키자 <br/>
앞에서 말했듯이 인풋과 아웃풋 차원은 어휘의 사이즈에 의해 정의된다. 임베딩 차원들과 인코더와 디코더의 드롭아웃은 다를 수 있지만 레이어와 은닉/셀 상태의 사이즈는 같아야한다.

In [16]:
INPUT_DIM = len(SRC.vocab)
OUTPUT_DIM = len(TRG.vocab)
ENC_EMB_DIM = 256
DEC_EMB_DIM = 256
HID_DIM = 512
N_LAYERS = 2
ENC_DROPOUT = 0.5
DEC_DROPOUT = 0.5

enc = Encoder(INPUT_DIM, ENC_EMB_DIM, HID_DIM, N_LAYERS, ENC_DROPOUT)
dec = Decoder(OUTPUT_DIM, DEC_EMB_DIM, HID_DIM, N_LAYERS, DEC_DROPOUT)

model = Seq2Seq(enc, dec, device).to(device)

그 다음단계는 모델의 가중치를 설정하는 것인데 논문에서는 모든 가중치를 -0.08에서 0.08 사이의 일정수치로 둔다고 한다. 

In [17]:
def init_weights(m):
    for name, param in m.named_parameters():
        nn.init.uniform_(param.data, -0.08,0.08)
        
model.apply(init_weights)

Seq2Seq(
  (encoder): Encoder(
    (embedding): Embedding(7853, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (dropout): Dropout(p=0.5)
  )
  (decoder): Decoder(
    (embedding): Embedding(5893, 256)
    (rnn): LSTM(256, 512, num_layers=2, dropout=0.5)
    (out): Linear(in_features=512, out_features=5893, bias=True)
  )
)

In [18]:
def count_parameters(model):
    return sum(p.numel() for p in model.parameters() if p.requires_grad)
print(f'The model has {count_parameters(model):,} trainable parameters')

The model has 13,898,501 trainable parameters


In [19]:
optimizer = optim.Adam(model.parameters())

이제 손실함수를 정의한다. 크로스엔트로피 손실 함수는 예측치에 대한 로그 소프트맥스 값뿐만
아니라 역의(negative) 소프트맥스 값까지 계산해준다.

이 손실함수는 토큰별로 평균손실값을 계산하지만 <pad>토큰의 인덱스에 대해서만 손실을 무시한다.

In [20]:
PAD_IDX = TRG.vocab.stoi['<pad>']
criterion = nn.CrossEntropyLoss(ignore_index= PAD_IDX)

In [21]:

def train(model, iterator, optimizer, criterion, clip):
    
    model.train()
    
    epoch_loss = 0
    
    for i, batch in enumerate(iterator):
        
        src = batch.src
        trg = batch.trg
        
        optimizer.zero_grad()
        
        output = model(src, trg)
        
        #trg = [trg sent len, batch size]
        #output = [trg sent len, batch size, output dim]
        
        output = output[1:].view(-1, output.shape[-1])
        trg = trg[1:].view(-1)
        
        #trg = [(trg sent len - 1) * batch size]
        #output = [(trg sent len - 1) * batch size, output dim]
        
        loss = criterion(output, trg)
        
        loss.backward()
        
        torch.nn.utils.clip_grad_norm_(model.parameters(), clip)
        
        optimizer.step()
        
        epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)


In [22]:
def evaluate(model, iterator, criterion):
    
    model.eval()
    
    epoch_loss = 0
    
    with torch.no_grad():
    
        for i, batch in enumerate(iterator):

            src = batch.src
            trg = batch.trg

            output = model(src, trg, 0) #turn off teacher forcing

            #trg = [trg sent len, batch size]
            #output = [trg sent len, batch size, output dim]

            output = output[1:].view(-1, output.shape[-1])
            trg = trg[1:].view(-1)

            #trg = [(trg sent len - 1) * batch size]
            #output = [(trg sent len - 1) * batch size, output dim]

            loss = criterion(output, trg)
            
            epoch_loss += loss.item()
        
    return epoch_loss / len(iterator)

다음은 한 에폭을 도는데 얼마나 걸리는지 알려주는함수다

In [23]:
def epoch_time (start_time, end_time):
    elapsed_time = end_time - start_time
    elapsed_mins = int(elapsed_time/60 )
    elapsed_secs = int(elased_time - (elapsed_mins*60))
    
    return elapsed_mins, elapsed_secs

각각의 에폭마다 우리 모델이 지금까지 validation loss를 얼마나 achieve 했는지 (?)
알고싶고 그때의 파라미터들을 저장하고싶다.<br/>
숫자가 커질수록 손실의 변화보다는 perplexity의 변화를 보여주는게 더 쉽기 때문에 각각의 에폭마다 loss와 perplexity를 프린트한다.

In [24]:
N_EPOCHS = 10
CLIP = 1

best_valid_loss = float('inf')

for epoch in range(N_EPOCHS):
    
    start_time = time.time()
    
    train_loss = train(model, train_iterator, optimizer, criterion, CLIP)
    valid_loss = evaluate(model, valid_iterator, criterion)
    
    end_time = time.time()
    
    epoch_mins, epoch_secs = epoch_time(start_time, end_time)
    
    if valid_loss < best_valid_loss:
        best_valid_loss = valid_loss
        torch.save(model.state_dict(), 'tut1-model.pt')
    
    print(f'Epoch: {epoch+1:02} | Time: {epoch_mins}m {epoch_secs}s')
    print(f'\tTrain Loss: {train_loss:.3f} | Train PPL: {math.exp(train_loss):7.3f}')
    print(f'\t Val. Loss: {valid_loss:.3f} |  Val. PPL: {math.exp(valid_loss):7.3f}')

TypeError: 'float' object is not callable